In [1]:
import numpy as np
np.random.seed(10)
import tensorflow as tf
tf.set_random_seed(10)
from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt

params = {
    'text.latex.preamble': ['\\usepackage{gensymb}'],
    'image.origin': 'lower',
    'image.interpolation': 'nearest',
    'image.cmap': 'gray',
    'axes.grid': False,
    'savefig.dpi': 300,  # to adjust notebook inline plot size
    'axes.labelsize': 16, # fontsize for x and y labels (was 10)
    'axes.titlesize': 16,
    'font.size': 16, # was 10
    'legend.fontsize': 16, # was 10
    'xtick.labelsize': 16,
    'ytick.labelsize': 16,
    'text.usetex': True,
    'figure.figsize': [3.39, 2.10],
    'font.family': 'serif',
}
plt.rcParams.update(params)

In [2]:
# Generating training data that goes from initial condition location to PCA coefficient trajectory
num_modes=40
locs = np.load('../../SWE_Data/Data/Locations.npy')
pca_coeffs = np.load('../../SWE_Data/PCA_Coefficients_q1.npy')[0:num_modes,:]

coeff_scaler = MinMaxScaler()
pca_coeffs_scaled = np.transpose(coeff_scaler.fit_transform(np.transpose(pca_coeffs)))

In [3]:
num_sims = np.shape(locs)[0]
num_ivs = np.shape(locs)[1]
num_coeffs = np.shape(pca_coeffs)[0]
burn_in = 20

In [4]:
# Reshape
training_data_ip = np.zeros(shape=(num_sims,num_coeffs+num_ivs,burn_in),dtype='double')
training_data_op = np.zeros(shape=(num_sims,num_coeffs+num_ivs,500-burn_in),dtype='double')

for sim in range(num_sims):
    training_data_ip[sim,:-num_ivs,:] = pca_coeffs_scaled[:,500*sim:500*sim+burn_in]
    training_data_ip[sim,-num_ivs:,:] = locs[sim,:,None]
    training_data_op[sim,:-num_ivs,:] = pca_coeffs_scaled[:,500*sim+burn_in:500*(sim+1)]
    training_data_op[sim,-num_ivs:,:] = locs[sim,:,None]

In [5]:
from tensorflow.keras.layers import Input, Dense, Lambda, Add, LSTM, Dropout, Bidirectional
from tensorflow.keras import optimizers, models, regularizers
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, Callback
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.regularizers import l1
from tensorflow.keras.utils import plot_model

In [6]:
weights_filepath = 'NA_BLSTM_P.h5'

def coeff_determination(y_pred, y_true): #Order of function inputs is important here        
    SS_res =  K.sum(K.square( y_true-y_pred )) 
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

class EarlyStoppingByLossVal(Callback):
    def __init__(self, monitor='loss', value=0.00001, verbose=0):
        super(Callback, self).__init__()
        self.monitor = monitor
        self.value = value
        self.verbose = verbose

    def on_epoch_end(self, epoch, logs={}):
        current = logs.get(self.monitor)
        if current is None:
            warnings.warn("Early stopping requires %s available!" % self.monitor, RuntimeWarning)

        if current < self.value:
            if self.verbose > 0:
                print("Epoch %05d: early stopping THR" % epoch)
            self.model.stop_training = True

In [10]:
lstm_inputs = Input(shape=(num_coeffs+num_ivs,burn_in),name='nat_inputs')
h1 = Bidirectional(LSTM(145,return_sequences=True))(lstm_inputs)
h1 = Dropout(0.0)(h1,training=True)
h2 = Bidirectional(LSTM(145,return_sequences=True))(h1)
h2 = Dropout(0.0)(h2,training=True)
lstm_outputs = Dense(500-burn_in,activation=None)(h2)

lstm_model = Model(inputs=lstm_inputs,outputs=lstm_outputs)
 
# design network
my_adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

checkpoint = ModelCheckpoint(weights_filepath, monitor='loss', verbose=1, save_best_only=True, mode='min',save_weights_only=True)
earlystopping = EarlyStopping(monitor='loss', min_delta=0, patience=100, verbose=0, mode='auto', baseline=None, restore_best_weights=False)
callbacks_list = [checkpoint,EarlyStoppingByLossVal()]

# fit network
lstm_model.compile(optimizer=my_adam,loss='mean_squared_error',metrics=[coeff_determination])    
lstm_model.summary()

train_mode = False

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
nat_inputs (InputLayer)      [(None, 42, 20)]          0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 42, 290)           192560    
_________________________________________________________________
dropout_2 (Dropout)          (None, 42, 290)           0         
_________________________________________________________________
bidirectional_3 (Bidirection (None, 42, 290)           505760    
_________________________________________________________________
dropout_3 (Dropout)          (None, 42, 290)           0         
_________________________________________________________________
dense_1 (Dense)              (None, 42, 480)           139680    
Total params: 838,000
Trainable params: 838,000
Non-trainable params: 0
_____________________________________________________

In [11]:
import matplotlib.pyplot as plt
lstm_model.load_weights(weights_filepath)

# Testing
filename = '../../SWE_Data/Data/snapshot_matrix_pod_test.npy'
test_data = np.load(filename)[0:64*64,:]
pca_vectors = np.load('../../SWE_Data/PCA_Vectors_q1.npy')[:64*64,:num_modes]

true_pca_evol = coeff_scaler.transform(np.matmul(np.transpose(test_data),pca_vectors))
test_data = np.zeros(shape=(1,num_coeffs+num_ivs,500))
test_data[0,0:num_coeffs,:] = np.transpose(true_pca_evol[:,:])

test_data[0,-2,:] = -1.0/2.7
test_data[0,-1,:] = -1.0/4.0

viz = False

mse_val = 0.0
num_inference = 1000
pred_mean = np.zeros_like(test_data)
pred_pca_array = np.zeros(shape=(num_inference,np.shape(test_data)[1],np.shape(test_data)[2]))

from time import time

start_time = time()

for inference in range(num_inference):
    
    pred_pca = lstm_model.predict(test_data[:,:,:burn_in])
    pred_pca = np.concatenate((test_data[:,:,:burn_in],pred_pca),axis=-1)
    
    pred_pca_array[inference,:,:] = pred_pca[0,:,:]
    mse_val = mse_val + np.sum((pred_pca-test_data)**2)
    pred_mean = pred_mean + pred_pca
    
    if viz:
    
        fig,ax = plt.subplots(nrows=2,ncols=2,figsize=(12,10))
        ax[0,0].plot(test_data[0,0,:],label='True')
        ax[0,0].plot(pred_pca[0,0,:],label='Predicted')
        ax[0,0].set_title('Mode 1')


        ax[1,0].plot(test_data[0,1,:],label='True')
        ax[1,0].plot(pred_pca[0,1,:],label='Predicted')
        ax[1,0].set_title('Mode 2')

        ax[0,1].plot(test_data[0,2,:],label='True')
        ax[0,1].plot(pred_pca[0,2,:],label='Predicted')
        ax[0,1].set_title('Mode 3')

        ax[1,1].plot(test_data[0,3,:],label='True')
        ax[1,1].plot(pred_pca[0,3,:],label='Predicted')
        ax[1,1].set_title('Mode 4')

        plt.tight_layout()
        plt.legend()
        plt.show()

        # Plotting some contours
        true_rb = np.transpose(coeff_scaler.inverse_transform(true_pca_evol))
        true_recon = np.matmul(pca_vectors,true_rb)[:,-2].reshape(64,64)

        pred_rb = np.transpose(pred_pca[0,:num_modes,:])
        pred_rb = np.transpose(coeff_scaler.inverse_transform(pred_rb))
        pred_recon = np.matmul(pca_vectors,pred_rb)[:,-2].reshape(64,64)


        fig,ax = plt.subplots(nrows=1,ncols=2,figsize=(12,5))
        cx = ax[0].contourf(true_recon)
        ax[1].contourf(pred_recon)

        fig.colorbar(cx,ax=ax[0],fraction=0.046, pad=0.04)
        fig.colorbar(cx,ax=ax[1],fraction=0.046, pad=0.04)
        plt.tight_layout()
        plt.show()
        
end_time = time()

In [12]:
print('Elapsed time per inference:',(end_time-start_time)/num_inference)

Elapsed time per inference: 0.016969022512435913


In [13]:
pred_mean = pred_mean/num_inference
pred_sdev = np.sqrt(np.sum((pred_pca_array - pred_mean[0,:,:])**2,axis=0)/(num_inference-1))
np.save('NA_BLSTM_P_0_Mean.npy',pred_mean)
np.save('NA_BLSTM_P_0_SD.npy',pred_sdev)